In [1]:
import cvxpy as cp
import numpy as np

from compas_pr2d import arch
from compas_pr2d.prd import PR2DModel

height = 4
span = 10
n = 12
thickness = 0.5

print("Constructing the model ...")
model = PR2DModel()
print("Constructing the arch geometry ...")
arch_geometry = arch.construct_arch(height, span, n, thickness)
print("Feeding the geometry to the model ...")

model.from_polygons(arch_geometry)
# model.display_mesh(bindex=True, nindex=True)

print("Assembling Matrices:")

model.set_boundary_edges(display=False)
disp_data = [
    [12, (-0.4,-0.4), (-0.0,-0.0)],
]  # [Interface ID, normal displacement, tangential displacement]


# model.set_force()  # for now no loads, just self-weight which is default in the model (i will add as gravity to all blocks for now only)
print("Assigning BCs to RHS vectors...")
model.assign_bc(disp_data)
# print("Solving the system...")
model.solve(dual=True, mu=0.3)
# fn_opt = model.results.fn
# ft_opt = model.results.ft
model.display_results(dual=False)
model.display_results(dual=True)
deformed = model.results.deformed_shape



Constructing the model ...
Constructing the arch geometry ...
Feeding the geometry to the model ...
Assembling Matrices:
Assembling boundary elements for edge: (0, 11, (0, 1))
Boundary edge rows: 22, 23
Assembling boundary elements for edge: (11, 12, (24, 25))
Boundary edge rows: 24, 25
Assigning BCs to RHS vectors...
Assigning displacement BC at interface 12 on edge (24, 25)
status: optimal
objective: -26.159042627806343
Showing results for primal problem
[ 2.14946058e-02  1.84267521e-02 -4.17357480e-02  6.66261755e-02
  4.00182117e-02 -4.17357480e-02  1.05805658e-01  7.11311965e-02
 -4.17357480e-02  1.37059456e-01  1.10198444e-01 -4.17357480e-02
  1.58813214e-01  1.55252009e-01 -4.17357480e-02  1.69971122e-01
  2.04022398e-01 -4.17357480e-02  1.69971122e-01  2.54052886e-01
 -4.17357480e-02  1.58813214e-01  3.02823274e-01 -4.17357480e-02
  1.37059456e-01  3.47876839e-01 -4.17357480e-02  1.07605851e-01
  3.82429952e-01 -3.39784333e-02  8.78048780e-02  3.90243902e-01
  1.89893685e-13  8

In [2]:

raise NotImplementedError("This is just a test file for the dual problem. The code is not meant to be run as is. It is just a reference for the implementation of the dual problem in the model.solve() method.")

A_ub = model.A_ub
A_eq = model.A_eq
U    = np.asarray(model.results.U).reshape(-1)
c    = np.asarray(model.c).reshape(-1)

m_ub, _ = A_ub.shape
m_eq, _ = A_eq.shape

# Stacking the dual problem data as in the antonino's paper
# As the CVXPY solver expects the objective function to be a single vector
# not a decoupled fn and ft.
delta_n = -model.b_ub
delta_t = -model.b_eq
# So all the data will be horitzontally stacked as follows:
# delta = np.hstack([A_ub @ U, A_eq @ U])          # (m_ub + m_eq,)
A_T    = np.hstack([A_ub.T,  A_eq.T])            # (n, m_ub + m_eq)

delta = np.hstack([delta_n, delta_t])            # (m_ub + m_eq,)
f = cp.Variable(m_ub + m_eq)

mu = 0.3
constraints = [
    A_T @ f == c,
    f[:m_ub] <= 0,
]
fn = f[:m_ub]
ft = f[m_ub:]

constraints += [
    ft <= -mu * (fn),
    ft >= mu * (fn),
]
objective = cp.Minimize(-(delta @ f))

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)
print("status:", prob.status)
print("objective:", prob.value)

f_opt = f.value                   

# Retrieveing the normal and tangential components of the force vector
fn_opt = f_opt[:m_ub]                # normal component
ft_opt = f_opt[m_ub:]                # tangential component

# Verify the optimality condition after decomposing the force vector.
res = A_ub.T @ fn_opt + A_eq.T @ ft_opt - c
print("||res||2 =", np.linalg.norm(res))


NotImplementedError: This is just a test file for the dual problem. The code is not meant to be run as is. It is just a reference for the implementation of the dual problem in the model.solve() method.

In [ ]:
print("fn:", fn_opt.reshape(-1,2))
print("ft:", ft_opt.reshape(-1,2))
print("fn shape:", fn_opt.shape, "ft shape:", ft_opt.shape)


edges = model.all_contacts
i = len(edges)
nodes = model.n

for edge in edges:
    print(f"Edge {i}: {edge} connects nodes {edge[0]} and {edge[1]}")
    i -= 1


fn: [[-2.82136727e+01 -2.93954677e+01]
 [-1.31695330e+01 -3.72382408e+01]
 [-1.08865812e+01 -3.25017818e+01]
 [-1.43490518e+01 -2.32511307e+01]
 [-1.80159692e+01 -1.58424000e+01]
 [-1.83705951e+01 -1.43058905e+01]
 [-1.42654234e+01 -1.99588527e+01]
 [-7.03688738e+00 -3.12766771e+01]
 [-3.81135645e-01 -4.40321488e+01]
 [-7.73164425e-10 -5.16926063e+01]
 [-1.30434445e+01 -4.60457181e+01]
 [-1.45639566e-09 -6.37969481e+01]
 [-4.73901986e+01 -1.80074079e+01]]
ft: [[-4.38078929e+00 -4.28177418e+00]
 [-1.57171255e-01 -1.53974276e+00]
 [-8.15789554e-05 -1.92747281e+00]
 [-6.32280698e-01 -2.08916972e+00]
 [-2.58030527e-01 -1.21763146e+00]
 [ 5.31093230e-01  2.89244243e-01]
 [-1.26050730e+00 -1.81450656e+00]
 [ 3.04474365e-01  3.89444028e+00]
 [ 2.00599539e-02 -3.22876623e+00]
 [-5.59523551e-11 -6.76610259e-01]
 [-3.74966822e-01 -7.57953705e+00]
 [ 2.66853916e-10  1.91390844e+01]
 [ 1.38078657e+01  4.97107058e+00]]
fn shape: (26,) ft shape: (26,)
Edge 13: (0, 1, 0, (2, 3)) connects nodes 0 and 

In [ ]:
import compas.geometry as cg
from compas.colors import Color
from compas_viewer import Viewer
from compas_viewer.scene import Tag


def weighted_point_on_segment(p0, p1, w0, w1):
    wsum = w0 + w1
    if wsum == 0:
        return cg.Point(0.5 * (p0.x + p1.x), 0.5 * (p0.y + p1.y), 0.5 * (p0.z + p1.z))
    return cg.Point(
        (w0 * p0.x + w1 * p1.x) / wsum,
        (w0 * p0.y + w1 * p1.y) / wsum,
        (w0 * p0.z + w1 * p1.z) / wsum,
    )

edges = model.all_contacts
nodes = model.n

points_t = []
viewer = Viewer()
# for shape in deformed:
#     viewer.scene.add(shape)
viewer.scene.add(model.mesh,opacity=0.5)
scale = 0.01
print(len(edges))
for bid,e in enumerate(edges):
    if len(e) == 4:
        i, j, k, (u, v) = e
    elif len(e) == 3:
        i, k, (u, v) = e
    else:
        continue

    A = cg.Point(*nodes[u])
    B = cg.Point(*nodes[v])
    line = cg.Line(A, B)

    # Get the normal force values at the endpoints of the edge

    fnA = float(fn_opt[2*k])   # end at node A
    fnB = float(fn_opt[2*k+1])   # end at node B

    # Weighted application point
    wA, wB = abs(fnA), abs(fnB)
    N = fnA + fnB
    pc = weighted_point_on_segment(A, B, wA, wB)

    t = cg.Vector(*line.vector)
    t.y = 0.0
    if t.length == 0:
        continue
    t.unitize()

    n = cg.Vector(-t.z, 0.0, t.x)
    if n.length == 0:
        continue
    n.unitize()

    # Magnitude (use average or resultant; here average)
    # tip = cg.Point(pc.x, pc.y, pc.z)
    # tip.translate(n * (scale * N))
    points_t.append([pc, N])
    # viewer.scene.add(cg.Line(pc, tip))   # <-- NO anchor here
    t = Tag(k.__str__(),position=pc)
    # viewer.scene.add(t)

points, N = zip(*points_t)
points = list(points)
polyline = cg.Polyline(points)

for point, n in zip(points, N):
    viewer.scene.add(point, vertexcolor=Color.from_number(n), pointsize=25 )
# viewer.scene.add(polyline)
viewer.show()


13
